In [1]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
# pretrained_model = "distilbert-base-uncased"
# pretrained_model = "xlm-roberta-large"
# pretrained_model = "microsoft/deberta-v3-small"
# pretrained_model = "microsoft/deberta-v3-base"
pretrained_model = "microsoft/deberta-v3-large"
# pretrained_model = "kamalkraj/deberta-v2-xlarge"
# model_checkpoint = "/root/ner-model/test-ner/checkpoint-2500/"

source_dataset_name = "wnut_17"
checkpoint_name = 'checkpoint-1278'
model_checkpoint = f"../data/models/{pretrained_model}/{source_dataset_name}/{checkpoint_name}"

use_pretrained_model = False ## vs. use checkpoint model
if use_pretrained_model:
    model_checkpoint = pretrained_model

batch_size = 24
num_train_epochs = 24

# dataset_name = "wnut_17"
# dataset_name = "conll2003"

dataset_name = "SpeedOfMagic/ontonotes_english"


In [2]:
import neptune.new as neptune
from transformers.integrations import NeptuneCallback

run_name = f'{pretrained_model}' if use_pretrained_model else f'{pretrained_model}/{source_dataset_name}'
tags = [pretrained_model, dataset_name, "NER"]
if not use_pretrained_model:
    tags.append(f'{source_dataset_name}->{dataset_name}')

# Create a Neptune run object
run = neptune.init_run(
    project="thingumajig/ner-training",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhN2QyZGRkNS0yNDhhLTQyYWItOTg5ZC1kZGQ3Yjc3ZDMxZTcifQ==",
    
    name=f"train-{run_name}:{dataset_name}", description=f"{run_name} fine-tuned on {dataset_name}",
    
    tags=tags,  # 

    source_files=["*.ipynb"],  # 

)

neptune_callback = NeptuneCallback(run=run)

neptune_run_id = run["sys/id"].fetch()
print(f'Run id: {neptune_run_id}')

2022-10-09 19:37:31.942690: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-09 19:37:32.068308: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2022-10-09 19:37:32.491674: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-09 19:37:32.491718: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-10-09 19:37:32.491724: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


https://app.neptune.ai/thingumajig/ner-training/e/NERTRAIN-6


Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


Run id: NERTRAIN-6


# Loading dataset

In [3]:
from datasets import load_dataset, load_metric

In [4]:
datasets = load_dataset(dataset_name)

Using custom data configuration SpeedOfMagic--ontonotes_english-955f75f6fda13fd9


Found cached dataset parquet (/root/.cache/huggingface/datasets/SpeedOfMagic___parquet/SpeedOfMagic--ontonotes_english-955f75f6fda13fd9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
datasets

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13900
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 8262
    })
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 59924
    })
})

In [6]:
datasets["train"][0]

{'tokens': ['Sunday', 'the', 'interview', 'with', 'Bob', 'Shapiro', '.'],
 'ner_tags': [15, 0, 0, 0, 1, 2, 0]}

In [7]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=37, names=['O', 'B-PERSON', 'I-PERSON', 'B-NORP', 'I-NORP', 'B-FAC', 'I-FAC', 'B-ORG', 'I-ORG', 'B-GPE', 'I-GPE', 'B-LOC', 'I-LOC', 'B-PRODUCT', 'I-PRODUCT', 'B-DATE', 'I-DATE', 'B-TIME', 'I-TIME', 'B-PERCENT', 'I-PERCENT', 'B-MONEY', 'I-MONEY', 'B-QUANTITY', 'I-QUANTITY', 'B-ORDINAL', 'I-ORDINAL', 'B-CARDINAL', 'I-CARDINAL', 'B-EVENT', 'I-EVENT', 'B-WORK_OF_ART', 'I-WORK_OF_ART', 'B-LAW', 'I-LAW', 'B-LANGUAGE', 'I-LANGUAGE'], id=None), length=-1, id=None)

In [8]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O',
 'B-PERSON',
 'I-PERSON',
 'B-NORP',
 'I-NORP',
 'B-FAC',
 'I-FAC',
 'B-ORG',
 'I-ORG',
 'B-GPE',
 'I-GPE',
 'B-LOC',
 'I-LOC',
 'B-PRODUCT',
 'I-PRODUCT',
 'B-DATE',
 'I-DATE',
 'B-TIME',
 'I-TIME',
 'B-PERCENT',
 'I-PERCENT',
 'B-MONEY',
 'I-MONEY',
 'B-QUANTITY',
 'I-QUANTITY',
 'B-ORDINAL',
 'I-ORDINAL',
 'B-CARDINAL',
 'I-CARDINAL',
 'B-EVENT',
 'I-EVENT',
 'B-WORK_OF_ART',
 'I-WORK_OF_ART',
 'B-LAW',
 'I-LAW',
 'B-LANGUAGE',
 'I-LANGUAGE']

In [9]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(datasets["train"])

,tokens,ner_tags
0,"[``, We, 're, confident, that, it, protects, our, route, structure, ,, '', he, says, ,, ``, and, our, ability, to, grow, and, prosper, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,"[Well, ,, Director, Yuan, ,, I, know, that, you, also, once, worked, in, Britain, .]","[O, O, O, B-PERSON, O, O, O, O, O, O, O, O, O, B-GPE, O]"
2,"[It, says, 40, %, of, those, surveyed, report, that, 1989, exploration, spending, will, exceed, 1988, 's, .]","[O, O, B-PERCENT, I-PERCENT, O, O, O, O, O, B-DATE, O, O, O, O, B-DATE, O, O]"
3,"[In, most, cases, ,, the, IRS, says, ,, these, projects, are, local, or, regional, ,, rather, than, national, ,, and, arise, because, auditors, in, an, area, detect, some, pattern, of, abuse, among, ,, say, ,, factory, workers, claiming, that, having, a, multitude, of, dependents, frees, them, from, tax, withholding, or, yacht, owners, deducting, losses, from, sideline, charter, businesses, .]","[O, O, O, O, O, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,"[Let, me, think, .]","[O, O, O, O]"
5,"[Congress, has, so, far, failed, to, provide, the, full, amount, that, the, White, House, has, sought, .]","[B-ORG, O, O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, O, O, O]"
6,"[While, residents, claimed, a, victory, ,, military, authorities, said, they, had, confiscated, the, equivalent, of, more, than, $, 1.5, million, to, make, up, for, the, unpaid, taxes, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-MONEY, I-MONEY, I-MONEY, I-MONEY, I-MONEY, O, O, O, O, O, O, O, O]"
7,"[Barbara, Platt, in, Aden, .]","[B-PERSON, I-PERSON, O, B-GPE, O]"
8,"[That, would, would, make, it, appealing, to, a, network, such, as, Interpublic, that, already, has, a, healthy, New, York, presence, .]","[O, O, O, O, O, O, O, O, O, O, O, B-ORG, O, O, O, O, O, B-GPE, I-GPE, O, O]"
9,"[According, to, a, spokesman, ,, the, governor, is, also, studying, the, possibility, of, raising, state, gasoline, taxes, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


# Preprocessing dataset

In [11]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [12]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [13]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [1, 5365, 261, 291, 269, 311, 4378, 300, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
example = datasets["train"][4]
print(example["tokens"])

['and', 'they', 'are', 'in', 'Atlanta', '.']


In [15]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '▁and', '▁they', '▁are', '▁in', '▁Atlanta', '▁.', '[SEP]']


In [16]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(6, 8)

In [17]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, None]


In [18]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

8 8


In [19]:
label_all_tokens = True

In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [21]:
tokenize_and_align_labels(datasets['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[1, 1333, 262, 2109, 275, 3877, 31922, 323, 2], [1, 1864, 9977, 3330, 264, 8458, 1016, 4023, 323, 2], [1, 279, 1376, 271, 47701, 281, 40618, 10637, 263, 6654, 15773, 323, 2], [1, 272, 382, 268, 6654, 277, 262, 595, 40618, 277, 262, 423, 323, 2], [1, 263, 306, 281, 267, 5100, 323, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 15, 0, 0, 0, 1, 2, 0, -100], [-100, 31, 31, 0, 0, 0, 0, 17, 0, -100], [-100, 0, 0, 0, 0, 0, 1, 2, 0, 1, 2, 0, -100], [-100, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 0, 9, 0, -100]]}

In [22]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/60 [00:00<?, ?ba/s]

# Fine-tuning the model

In [23]:
from transformers import AutoConfig, AutoModelForTokenClassification, TrainingArguments, Trainer
# config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), ignore_mismatched_sizes=True) #, from_tf=True

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at ../data/models/microsoft/deberta-v3-large/wnut_17/checkpoint-1278 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([13, 1024]) in the checkpoint and torch.Size([37, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([13]) in the checkpoint and torch.Size([37]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
args = TrainingArguments(
    # f"test-{task}",
    f"../data/models/{pretrained_model}/{dataset_name}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    weight_decay=0.01,
    num_train_epochs=num_train_epochs,
    
    save_strategy='epoch',
    save_total_limit = 3,
    
    metric_for_best_model = 'f1',
    load_best_model_at_end=True,
    
    report_to="none",
)

In [25]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [26]:
metric = load_metric("seqeval")
# from evaluate import evaluator

# task_evaluator = evaluator("token-classification")

/tmp/ipykernel_2607011/1092872775.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [27]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'GPE': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [28]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Trainer

In [29]:
import torch
torch.cuda.empty_cache()

In [30]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    
    callbacks=[neptune_callback],
)

In [31]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.


/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****


  Num examples = 59924


  Num Epochs = 24


  Instantaneous batch size per device = 24


...

Waiting for the remaining 7 operations to synchronize with Neptune. Do not kill this process.


All 7 operations synced, thanks for waiting!


Explore the metadata in the Neptune app:


https://app.neptune.ai/thingumajig/ner-training/e/NERTRAIN-6


TrainOutput(global_step=59928, training_loss=0.011914202789777893, metrics={'train_runtime': 20168.0692, 'train_samples_per_second': 71.31, 'train_steps_per_second': 2.971, 'total_flos': 1.554383065463742e+17, 'train_loss': 0.011914202789777893, 'epoch': 24.0})

In [32]:
restored_run = neptune.init_run(
    with_id=neptune_run_id,
    
    project="thingumajig/ner-training",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhN2QyZGRkNS0yNDhhLTQyYWItOTg5ZC1kZGQ3Yjc3ZDMxZTcifQ==",
)
trainer.pop_callback(neptune_callback)
trainer.add_callback(NeptuneCallback(run=restored_run))

trainer.evaluate()

https://app.neptune.ai/thingumajig/ner-training/e/NERTRAIN-6


Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.


***** Running Evaluation *****


  Num examples = 13900


  Batch size = 24


/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:745: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:829: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  score += c2p_att / torch.tensor(scale, dtype=c2p_att.dtype)
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:

{'eval_loss': 0.29370418190956116,
 'eval_precision': 0.8376308853906973,
 'eval_recall': 0.9130755493438931,
 'eval_f1': 0.8737276172078163,
 'eval_accuracy': 0.9716869709401597,
 'eval_runtime': 53.3844,
 'eval_samples_per_second': 260.376,
 'eval_steps_per_second': 10.865,
 'epoch': 24.0}

In [33]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.


***** Running Prediction *****


  Num examples = 13900


  Batch size = 24


Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: finetuning/eval/recall. Invalid point: 59928.0


...

Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: finetuning/eval/runtime. Invalid point: 59928.0


Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: finetuning/eval/steps_per_second. Invalid point: 59928.0


Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: finetuning/eval/f1. Invalid point: 59928.0


Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: finetuning/eval/loss. Invalid point: 59928.0


{'CARDINAL': {'precision': 0.7750851950018932,
  'recall': 0.911804008908686,
  'f1': 0.8379042161277118,
  'number': 2245},
 'DATE': {'precision': 0.8352320088913587,
  'recall': 0.8951756998213222,
  'f1': 0.8641655886157826,
  'number': 3358},
 'EVENT': {'precision': 0.6057692307692307,
  'recall': 0.6146341463414634,
  'f1': 0.6101694915254237,
  'number': 205},
 'FAC': {'precision': 0.5046728971962616,
  'recall': 0.7448275862068966,
  'f1': 0.6016713091922006,
  'number': 145},
 'GPE': {'precision': 0.9069243760555451,
  'recall': 0.9525029562475364,
  'f1': 0.9291550514274728,
  'number': 5074},
 'LANGUAGE': {'precision': 0.9393939393939394,
  'recall': 0.8157894736842105,
  'f1': 0.8732394366197183,
  'number': 38},
 'LAW': {'precision': 0.5227272727272727,
  'recall': 0.6666666666666666,
  'f1': 0.5859872611464967,
  'number': 69},
 'LOC': {'precision': 0.7242206235011991,
  'recall': 0.7803617571059431,
  'f1': 0.7512437810945274,
  'number': 387},
 'MONEY': {'precision': 0.9

In [34]:
# trainer.log(results)
restored_run['evaluation_extra'] = results

In [35]:
restored_run.stop()


Shutting down background jobs, please wait a moment...


Done!


Waiting for the remaining 80 operations to synchronize with Neptune. Do not kill this process.


All 80 operations synced, thanks for waiting!


Explore the metadata in the Neptune app:


https://app.neptune.ai/thingumajig/ner-training/e/NERTRAIN-6
